In [5]:
import boto3

s3 = boto3.client('s3')

s3 = boto3.resource(

        service_name = 's3',
        region_name = 'us-east-2',
        aws_access_key_id = aws_access_key_id,
        aws_secret_access_key = aws_secret_access_key
    
)

for object in s3.Bucket('newbucketformyproject').objects.all():
    print(object.key)


IDS_mapping.csv
diabetes_012_health_indicators_BRFSS2015.csv
diabetes_binary_5050split_health_indicators_BRFSS2015.csv
diabetes_binary_health_indicators_BRFSS2015.csv
diabetic_data.csv
drugsComTest_raw.tsv
drugsComTrain_raw.tsv


In [6]:
# Set the Hadoop configurations for S3 access
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key_id)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_access_key)


In [10]:
drugsComTest_raw = spark.read.csv("s3a://newbucketformyproject/drugsComTest_raw.tsv", sep='\t', header=True)
drugsComTest_raw.show()

+------+--------------------+--------------------+--------------------+------+------------------+-----------+
|   _c0|            drugName|           condition|              review|rating|              date|usefulCount|
+------+--------------------+--------------------+--------------------+------+------------------+-----------+
|163740|         Mirtazapine|          Depression|"""I&#039;ve trie...|  10.0| February 28, 2012|         22|
|206473|          Mesalamine|Crohn's Disease, ...|"""My son has Cro...|   8.0|      May 17, 2009|         17|
|159672|             Bactrim|Urinary Tract Inf...|"""Quick reductio...|   9.0|September 29, 2017|          3|
| 39293|            Contrave|         Weight Loss|"""Contrave combi...|   9.0|     March 5, 2017|         35|
| 97768|     Cyclafem 1 / 35|       Birth Control|"""I have been on...|   9.0|  October 22, 2015|          4|
|208087|             Zyclara|           Keratosis|"""4 days in on f...|   4.0|      July 3, 2014|         13|
|215892|  

In [12]:
drugsComTest_raw

DataFrame[_c0: string, drugName: string, condition: string, review: string, rating: string, date: string, usefulCount: string]

In [25]:
# Reading TSV file into a Spark DataFrame
drugsComTrain_raw = spark.read.csv("s3a://newbucketformyproject/drugsComTrain_raw.tsv", sep='\t', header=True)
drugsComTrain_raw.show()

+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|                 _c0|            drugName|           condition|              review|rating|             date|usefulCount|
+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|              206461|           Valsartan|Left Ventricular ...|"""It has no side...|   9.0|     May 20, 2012|         27|
|               95260|          Guanfacine|                ADHD|"""My son is half...|  NULL|             NULL|       NULL|
|We have tried man...|                 8.0|      April 27, 2010|                 192|  NULL|             NULL|       NULL|
|               92703|              Lybrel|       Birth Control|"""I used to take...|  NULL|             NULL|       NULL|
|The positive side...|                 5.0|   December 14, 2009|                  17|  NULL|             NULL|       NULL|
|              1

In [3]:
drugsComTrain_raw

NameError: name 'drugsComTrain_raw' is not defined

In [29]:
drugsComTrain_raw = drugsComTrain_raw.union(drugsComTest_raw)
drugsComTrain_raw.show()

+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|                 _c0|            drugName|           condition|              review|rating|             date|usefulCount|
+--------------------+--------------------+--------------------+--------------------+------+-----------------+-----------+
|              206461|           Valsartan|Left Ventricular ...|"""It has no side...|   9.0|     May 20, 2012|         27|
|               95260|          Guanfacine|                ADHD|"""My son is half...|  NULL|             NULL|       NULL|
|We have tried man...|                 8.0|      April 27, 2010|                 192|  NULL|             NULL|       NULL|
|               92703|              Lybrel|       Birth Control|"""I used to take...|  NULL|             NULL|       NULL|
|The positive side...|                 5.0|   December 14, 2009|                  17|  NULL|             NULL|       NULL|
|              1

In [33]:
drugsComTrain_raw.dropDuplicates().show()

+------+--------------------+--------------------+--------------------+------+------------------+-----------+
|   _c0|            drugName|           condition|              review|rating|              date|usefulCount|
+------+--------------------+--------------------+--------------------+------+------------------+-----------+
|126595|         Brimonidine|             Rosacea|"""This cream is ...|   1.0|  January 31, 2017|         16|
|198429|           Exenatide|    Diabetes, Type 2|"""I was on Bydur...|   6.0|      May 28, 2013|         34|
|145101|        Etonogestrel|       Birth Control|"""I switched to ...|   5.0|  November 7, 2015|          1|
| 80302|Alavert D-12 Hour...|1</span> users fo...|"""I&#039;ve foun...|   5.0|    April 15, 2016|          1|
|142866|        Etonogestrel|       Birth Control|"""My arm was bru...|   1.0|September 22, 2017|          2|
| 46156|             Repatha|    High Cholesterol|"""I have always ...|   1.0| December 27, 2016|         59|
|217342|  

In [39]:
drugsComTrain_raw.count()

329028

In [40]:
drugsComTest_raw.count()

65987